In [4]:
%load_ext griblib.jupyter

from pathlib import Path
from warnings import warn
from datetime import datetime
from typing import Callable, Union, Iterable, Iterator, Literal, TypedDict

import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.dataframe.core import DataFrame as DaskDataFrame
from geopandas import GeoDataFrame
from requests import Session, HTTPError
from typing import Generic, TypeVar
from geopandas.base import GeometryArray
from geopandas import GeoSeries
import geopandas._vectorized as geov
import pygeos
from numpy.typing import NDArray

import xarray as xr
from shapely.geometry import GeometryCollection

import warnings

/opt/conda/envs/rapids/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [5]:
class Properties(TypedDict):
    MUCAPE: int
    MLCAPE: int
    MLCIN: int
    EBSHEAR: float
    SRH01KM: int
    MESH: float
    VIL_DENSITY: float
    FLASH_RATE: int
    FLASH_DENSITY: float
    MAXLLAZ: float
    P98LLAZ: float
    P98MLAZ: float
    MAXRC_EMISS: str
    MAXRC_ICECF: str
    WETBULB_0C_HGT: float
    PWAT: float
    CAPE_M10M30: int
    LJA: float
    SIZE: int
    AVG_BEAM_HGT: str
    MOTION_EAST: float
    MOTION_SOUTH: float
    PS: int
    ID: int


class Geometry(TypedDict):
    type: Literal["Polygon"]
    coordinates: list[list[tuple[float, float]]]


class Feature(TypedDict):
    type: Literal["Feature"]
    geometry: Geometry
    models: dict[str, dict[str, str]]
    properties: Properties


class FeatureCollection(TypedDict):
    source: Literal["NOAA/NCEP Central Operations"]
    product: Literal["ProbSevere"]
    type: Literal["FeatureCollection"]
    validTime: str
    productionTime: str
    machine: str
    features: list[Feature]


TimeLike = Union[datetime, str, pd.Timestamp]

In [17]:
import requests
import xarray as xr

VALIDTIME_TEMPLATE = "%Y%m%d_%H%M%S %Z"
FLOAT32_COLS = [
    "EBSHEAR",
    "MEANWIND_1-3kmAGL",
    "MESH",
    "VIL_DENSITY",
    "FLASH_DENSITY",
    "MOTION_EAST",
    "MOTION_SOUTH",
    "MAXLLAZ",
    "P98LLAZ",
    "P98MLAZ",
    "WETBULB_0C_HGT",
    "PWAT",
    "LJA",
]

INT32_COLS = ["MLCIN"]
UINT32_COLS = [
    "MUCAPE",
    "MLCAPE",
    "SRH01KM",
    "FLASH_RATE",
    "CAPE_M10M30",
    "SIZE",
    "ID",
]
UINT8_COLS = ["PS"]
ALL_COLUMNS = UINT8_COLS + UINT32_COLS + INT32_COLS + FLOAT32_COLS


def extract(date: TimeLike) -> Iterable[FeatureCollection]:
    base_url = "https://mtarchive.geol.iastate.edu"
    url = f"{base_url}/{date:%Y}/{date:%m}/{date:%d}/mrms/ncep/ProbSevere/"
    r = requests.get(url)
    if r.status_code == 200:
        (df,) = pd.read_html(r.text, skiprows=[1, 2], keep_default_na=False)

        with Session() as session:
            for file in tuple(url + df.loc[df["Name"] != "", "Name"]):
                try:
                    # with our session make a get request, r is a response object
                    r = session.get(file, stream=True)
                    # in the event of a non 200 status code we'll raise a HTTPError and trigger the except block
                    r.raise_for_status()
                # if there was an error downloading, continue
                except (ConnectionError, HTTPError):
                    warn(f"error downloading {url}")
                    continue
                yield r.json()


def _wrangle_geometry(df: GeoDataFrame) -> pd.DataFrame:
    # to keep things consistent uppercase all of the bounds
    bounds = df.bounds
    df[bounds.columns.str.upper()] = bounds
    with warnings.catch_warnings():
        # /opt/conda/envs/rapids/lib/python3.9/site-packages/geopandas/array.py:524:
        # ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
        #   return GeometryArray(vectorized.representative_point(self.data), crs=self.crs)
        warnings.simplefilter("ignore")
        points = df.representative_point()
    df["Y"] = points.x
    df["X"] = points.y
    return df


def update_dtypes(df: pd.DataFrame, float32: list[str], int32: list[str], uint32: list[str], uint8: list[str]):
    df[float32] = df[float32].astype(np.float32)
    # 32-bit signed integer (``-2_147_483_648`` to ``2_147_483_647``)
    df[int32] = df[int32].astype(np.int32)
    # 32-bit unsigned integer (``0`` to ``4_294_967_295``)
    df[uint32] = df[uint32].astype(np.uint32)
    # numpy.uint8`: 8-bit unsigned integer (``0`` to ``255``)
    df[uint8] = df[uint8].astype(np.uint8)
    return df


def transfer(data: Iterable[FeatureCollection]) -> xr.Dataset:
    keys = []

    def generate():
        for fc in data:
            keys.append(datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE))
            df = GeoDataFrame.from_features(fc["features"], columns=ALL_COLUMNS + ["geometry"])
            df["time"] = datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE)
            yield df

    df = pd.concat(generate()).set_index("time").pipe(_wrangle_geometry).drop(columns="geometry")

    df = update_dtypes(
        df,
        float32=FLOAT32_COLS + ["MINX", "MINY", "MAXX", "MAXY", "X", "Y"],
        int32=INT32_COLS,
        uint32=UINT32_COLS,
        uint8=UINT8_COLS,
    )
    return xr.Dataset.from_dataframe(df)


def load(ds: xr.Dataset, store: Path) -> None:
    zarr_kwargs = {}
    if store.exists():
        zarr_kwargs = {"mode": "a", "append_dim": "time"}
    ds.to_zarr(store, **zarr_kwargs)


def main():
    store = Path.cwd().parent / "data/PROBSEVERE"
    date = datetime.fromisoformat("2022-01-01")
    data = extract(date)
    ds = transfer(data)
    load(ds, store)


main()

In [19]:
xr.open_zarr(Path.cwd().parent / "data/PROBSEVERE").to_dataframe().set_index("ID", append=True)

CAPE_M10M30    EBSHEAR  FLASH_DENSITY  FLASH_RATE  \
time                ID                                                         
2022-01-01 00:00:38 20964          207  49.700001           0.02           0   
                    21312          187  43.700001           0.00           0   
                    21320            0   0.000000           0.00           0   
                    21323          169  43.200001           0.00           0   
                    21324            0   0.000000           0.00           0   
...                                ...        ...            ...         ...   
2022-01-01 23:58:38 28342            0  27.000000           0.00           0   
                    28343           41  59.299999           0.00           0   
                    28344           85  61.799999           0.00           0   
                    28345          248  56.700001           0.00           0   
                    28346          478  65.599998           0.00           0   

                           LJA  MAXLLAZ        MAXX       MAXY  \
time                ID                                           
2022-01-01 00:00:38 20964  0.0    0.005  -83.879997  34.730000   
                    21312  0.0    0.004 -110.470001  32.799999   
                    21320  0.0    0.000  -71.029999  47.990002   
                    21323  0.0    0.001 -110.320000  32.939999   
                    21324  0.0    0.000  -95.980003  39.849998   
...                        ...      ...         ...        ...   
2022-01-01 23:58:38 28342  0.0    0.001  -88.330002  36.230000   
                    28343  0.0    0.004  -85.620003  35.849998   
                    28344  0.0    0.013  -85.669998  35.619999   
                    28345  0.0    0.002  -87.879997  34.840000   
                    28346  0.0    0.002  -93.120003  32.529999   

                           MEANWIND_1-3kmAGL  MESH  ...  P98LLAZ  P98MLAZ  PS  \
time                ID                              ...                         
2022-01-01 00:00:38 20964          36.799999  0.04  ...    0.004    0.004   8   
                    21312          34.500000  0.00  ...    0.003    0.003   3   
                    21320          31.200001  0.18  ...    0.000    0.000   0   
                    21323          35.099998  0.00  ...    0.001    0.001   3   
                    21324          35.500000  0.27  ...    0.000    0.000   0   
...                                      ...   ...  ...      ...      ...  ..   
2022-01-01 23:58:38 28342          27.500000  0.00  ...    0.001    0.001   0   
                    28343          50.700001  0.00  ...    0.004    0.006   4   
                    28344          52.799999  0.03  ...    0.012    0.005   6   
                    28345          46.200001  0.00  ...    0.002    0.001   5   
                    28346          31.900000  0.00  ...    0.002    0.002   6   

                           PWAT  SIZE  SRH01KM  VIL_DENSITY  WETBULB_0C_HGT  \
time                ID                                                        
2022-01-01 00:00:38 20964   1.4   306      200         0.79             9.1   
                    21312   0.7   128       85         0.37             4.2   
                    21320   0.3    68      168         0.94             0.4   
                    21323   0.7   135       84         0.41             3.5   
                    21324   0.4    69      150         0.51             0.3   
...                         ...   ...      ...          ...             ...   
2022-01-01 23:58:38 28342   1.4    45        0         0.38            11.3   
                    28343   1.6    49      170         1.01            10.9   
                    28344   1.6    43      289         1.21            10.0   
                    28345   1.8    46      171         0.24            11.3   
                    28346   1.7    43       93         0.50            10.8   

                                   X         

In [81]:
URL_TEMPLATE = "https://mtarchive.geol.iastate.edu/%Y/%m/%d/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_%Y%m%d_%H%M00.json"
VALIDTIME_TEMPLATE = "%Y%m%d_%H%M%S %Z"

def iterdaterange(
    start: TimeLike, end: TimeLike, *, freq: str = "2min"
) -> Iterator[tuple[pd.Timestamp, pd.DataFrame]]:

    dr = pd.date_range(start=start, end=end, freq=freq)
    urls = dr.strftime(URL_TEMPLATE)
    yield from pd.DataFrame({"date": dr, "urls": urls}).set_index(dr).groupby(pd.Grouper(key="date", freq="D", axis=0))


def generate_from_features(session: Session, *, urls: Iterable[str]) -> Iterable[pd.DataFrame]:
    for url in urls:
        try:
            # with our session make a get request, r is a response object
            r = session.get(url, stream=True)
            # in the event of a non 200 status code we'll raise a HTTPError and trigger the except block
            r.raise_for_status()
        # if there was an error downloading, continue
        except (ConnectionError, HTTPError):
            warn(f"error downloading {url}")
            continue
        fc: FeatureCollection = r.json()

        features = fc["features"]
        # in the event no storms were record, continue
        if not features:
            # geojson file contained no features
            continue

        df = GeoDataFrame.from_features(features)
        df["VALIDTIME"] = datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE)
        yield df


def wrangle_geometry(df: GeoDataFrame) -> pd.DataFrame:
    # to keep things consistent uppercase all of the bounds
    bounds = df.bounds
    df[bounds.columns.str.upper()] = bounds
    point = df.representative_point()
    df["X"] = point.x
    df["Y"] = point.y
    return df


def __wrangle_dtypes(
    ddf: DaskDataFrame,
) -> DaskDataFrame:
    float32_cols = [
        "EBSHEAR",
        "MEANWIND_1-3kmAGL",
        "MESH",
        "VIL_DENSITY",
        "FLASH_DENSITY",
        "MOTION_EAST",
        "MOTION_SOUTH",
        "MAXLLAZ",
        "P98LLAZ",
        "P98MLAZ",
        "WETBULB_0C_HGT",
        "PWAT",
        "LJA",
        "MINX",
        "MINY",
        "MAXX",
        "MAXY",
        "X",
        "Y",
    ]
    int32_cols = [
        "MLCIN",
    ]
    uint32_cols = [
        "MUCAPE",
        "MLCAPE",
        "SRH01KM",
        "FLASH_RATE",
        "CAPE_M10M30",
        "SIZE",
        "ID",
    ]
    # 0 - 255
    uint8_cols = [
        "PS",
    ]

    ddf[float32_cols] = ddf[float32_cols].astype(np.float32)
    # 32-bit signed integer (``-2_147_483_648`` to ``2_147_483_647``)
    ddf[int32_cols] = ddf[int32_cols].astype(np.int32)
    # 32-bit unsigned integer (``0`` to ``4_294_967_295``)
    ddf[uint32_cols] = ddf[uint32_cols].astype(np.uint32)
    # numpy.uint8`: 8-bit unsigned integer (``0`` to ``255``)
    ddf[uint8_cols] = ddf[uint8_cols].astype(np.uint8)
    return ddf


def to_dask(data: pd.DataFrame, *, chunk_size: int) -> DaskDataFrame:
    return dd.from_pandas(data, chunksize=chunk_size).pipe(wrangle_dtypes)  # type: ignore



def extract(
    path: Path,
    *,
    start: TimeLike,
    end: TimeLike,
    freq: str = "2min",
    chunk_size: int = 256,
) -> None:
    """download and save probsevere data

    Parameters
    ----------
    path : Path
        path to where the file should be save
    start : str or datetime-like
        Left bound for generating dates.
    end : str or datetime-like
        Right bound for generating dates.
    freq : str
        string to step the urls
    name_function : `(n:int) -> str`
        ...
    chunk_size : int; defualt 256
        passed to dask, ...
    """
    drop_columns = ["MAXRC_EMISS", "MAXRC_ICECF", "AVG_BEAM_HGT", "geometry"]
    with Session() as session:
        for timestamp, values in iterdaterange(start, end, freq=freq):
            # create the inital pandas dataframe
            yield (
                # download data
                pd.concat(generate_from_features(session, urls=values["urls"]))
                # wrangle the geometry
                .pipe(wrangle_geometry)
                .drop(columns=drop_columns)
                .pipe(__to_dask, chunk_size=chunk_size)
                # .to_parquet(  # type: ignore
                #     path,
                #     engine="pyarrow",
                #     append=True,
                #     name_function=__name_function(timestamp),
                #     ignore_divisions=True,
                # )
            )


In [75]:
import requests

VALIDTIME_TEMPLATE = "%Y%m%d_%H%M%S %Z"
data = []
# x = []
base_url = "https://mtarchive.geol.iastate.edu"
# with Session() as session:
# for date in pd.date_range("2022-01-01T00:00", "2022-01-01T00:02", freq="d"):
date = datetime.fromisoformat("2022-01-01")
url = f"{base_url}/{date:%Y}/{date:%m}/{date:%d}/mrms/ncep/ProbSevere/"
r = requests.get(url)
(df,) = pd.read_html(r.text, skiprows=[1, 2], keep_default_na=False)
with Session() as session:
    for file in tuple(url + df.loc[df["Name"] != "", "Name"])[:2]:
        try:
            # with our session make a get request, r is a response object
            r = session.get(file, stream=True)
            # in the event of a non 200 status code we'll raise a HTTPError and trigger the except block
            r.raise_for_status()
        # if there was an error downloading, continue
        except (ConnectionError, HTTPError):
            warn(f"error downloading {url}")
            continue
        fc: FeatureCollection = r.json()
        data.append(fc)
        # r = session.get(url, stream=True)
        # ...
    # print(pd.read_html)
    # r = request
    # print(f"{url}MRMS_PROBSEVERE_{date:%Y}{date:%m}{date:%d}_{date:%H}{date:%M}00.json")
    # url =

    # features = fc["features"]
    # # in the event no storms were record, continue
    # if not features:
    #     # geojson file contained no features
    #     continue

    # df = GeoDataFrame.from_features(features)
    # df["VALIDTIME"] = datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE)
    # yield df
data

[{'source': 'NOAA/NCEP Central Operations',
  'product': 'ProbSevere',
  'validTime': '20220101_000038 UTC',
  'productionTime': '20220101_000219 UTC',
  'machine': 'vm-bldr-mrms-ops-probsvr1.ncep.noaa.gov',
  'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[-83.99, 34.73],
       [-83.93, 34.73],
       [-83.91, 34.72],
       [-83.88, 34.69],
       [-83.88, 34.65],
       [-83.89, 34.63],
       [-83.91, 34.62],
       [-83.93, 34.6],
       [-83.93, 34.59],
       [-83.97, 34.57],
       [-84.01, 34.57],
       [-84.02, 34.56],
       [-84.04, 34.56],
       [-84.08, 34.54],
       [-84.11, 34.54],
       [-84.12, 34.55],
       [-84.12, 34.6],
       [-84.13, 34.61],
       [-84.11, 34.62],
       [-84.08, 34.62],
       [-84.04, 34.66],
       [-84.02, 34.71],
       [-84.01, 34.71],
       [-83.99, 34.73]]]},
    'models': {'probsevere': {'PROB': '2',
      'LINE01': 'ProbHail: 1%; ProbWind: 1%; ProbTor: 2

In [337]:

from typing import Generic, TypeVar
from geopandas.base import GeometryArray
from geopandas import GeoSeries
import geopandas._vectorized as geov
import pygeos
from numpy.typing import NDArray


from shapely.geometry import GeometryCollection

import warnings

# np.read_html()
float32_cols = [
    "EBSHEAR",
    "MEANWIND_1-3kmAGL",
    "MESH",
    "VIL_DENSITY",
    "FLASH_DENSITY",
    "MOTION_EAST",
    "MOTION_SOUTH",
    "MAXLLAZ",
    "P98LLAZ",
    "P98MLAZ",
    "WETBULB_0C_HGT",
    "PWAT",
    "LJA",
    # "MINX",
    # "MINY",
    # "MAXX",
    # "MAXY",
    # "X",
    # "Y",
]
int32_cols = [
    "MLCIN",
]
uint32_cols = [
    "MUCAPE",
    "MLCAPE",
    "SRH01KM",
    "FLASH_RATE",
    "CAPE_M10M30",
    "SIZE",
    "ID",
]
# 0 - 255
uint8_cols = [
    "PS",
]
all_columns = uint8_cols + uint32_cols + int32_cols + float32_cols



def wrangle_geometry(df: GeoDataFrame) -> pd.DataFrame:
    # to keep things consistent uppercase all of the bounds
    bounds = df.bounds
    df[bounds.columns.str.upper()] = bounds
    with warnings.catch_warnings():
        # /opt/conda/envs/rapids/lib/python3.9/site-packages/geopandas/array.py:524:
        # ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
        #   return GeometryArray(vectorized.representative_point(self.data), crs=self.crs)
        warnings.simplefilter("ignore")
        points = df.representative_point()
    df["Y"] = points.x
    df["X"] = points.y
    return df


def dtypes(ddf: pd.DataFrame):
    float32_cols = [
        "EBSHEAR",
        "MEANWIND_1-3kmAGL",
        "MESH",
        "VIL_DENSITY",
        "FLASH_DENSITY",
        "MOTION_EAST",
        "MOTION_SOUTH",
        "MAXLLAZ",
        "P98LLAZ",
        "P98MLAZ",
        "WETBULB_0C_HGT",
        "PWAT",
        "LJA",
        "MINX",
        "MINY",
        "MAXX",
        "MAXY",
        "X",
        "Y",
    ]
    int32_cols = [
        "MLCIN",
    ]
    uint32_cols = [
        "MUCAPE",
        "MLCAPE",
        "SRH01KM",
        "FLASH_RATE",
        "CAPE_M10M30",
        "SIZE",
        "ID",
    ]
    # 0 - 255
    uint8_cols = [
        "PS",
    ]

    ddf[float32_cols] = ddf[float32_cols].astype(np.float32)
    # 32-bit signed integer (``-2_147_483_648`` to ``2_147_483_647``)
    ddf[int32_cols] = ddf[int32_cols].astype(np.int32)
    # 32-bit unsigned integer (``0`` to ``4_294_967_295``)
    ddf[uint32_cols] = ddf[uint32_cols].astype(np.uint32)
    # numpy.uint8`: 8-bit unsigned integer (``0`` to ``255``)
    ddf[uint8_cols] = ddf[uint8_cols].astype(np.uint8)
    return ddf


import xarray as xr


def transfer(data: Iterable[FeatureCollection]) -> xr.Dataset:
    keys = []

    def generate():
        for fc in data:
            keys.append(datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE))
            df = GeoDataFrame.from_features(fc["features"], columns=all_columns + ["geometry"])
            df["time"] = datetime.strptime(fc["validTime"], VALIDTIME_TEMPLATE)
            yield df

    df = pd.concat(generate()).set_index("time").pipe(wrangle_geometry).drop(columns="geometry").pipe(dtypes)

    return xr.Dataset.from_dataframe(df)


#
store = Path.cwd().parent / "data/PROBSEVERE"


def main():
    ds = transfer(data)
    zarr_kwargs = {}
    if store.exists():
        zarr_kwargs = {"mode": "a", "append_dim": "time"}

    ds.to_zarr(store, **zarr_kwargs)


main()

In [339]:
xr.open_zarr(store).to_dataframe().set_index("ID", append=True)

CAPE_M10M30    EBSHEAR  FLASH_DENSITY  FLASH_RATE  \
time                ID                                                         
2022-01-01 00:00:38 20964          207  49.700001           0.02           0   
                    21312          187  43.700001           0.00           0   
                    21320            0   0.000000           0.00           0   
                    21323          169  43.200001           0.00           0   
                    21324            0   0.000000           0.00           0   
                    21328            0   0.000000           0.00           0   
                    21329            0  23.100000           0.00           0   
                    21330           74  23.400000           0.00           0   
                    21331          116  31.700001           0.00           0   
                    21332          196  52.400002           0.00           0   
                    21333           54  38.900002           0.00           0   
2022-01-01 00:02:44 20964          207  49.700001           0.02           0   
                    21312          187  43.700001           0.00           0   
                    21320            0   0.000000           0.00           0   
                    21323          169  43.200001           0.00           0   
                    21324            0   0.000000           0.00           0   
                    21328            0   0.000000           0.00           0   
                    21330           74  23.400000           0.00           0   
                    21331          116  31.799999           0.00           0   
                    21332          194  51.700001           0.00           0   
                    21333           54  39.000000           0.00           0   
                    21334            0   0.000000           0.00           0   
                    21335          153  46.700001           0.03           0   

                           LJA  MAXLLAZ        MAXX       MAXY  \
time                ID                                           
2022-01-01 00:00:38 20964  0.0    0.005  -83.879997  34.730000   
                    21312  0.0    0.004 -110.470001  32.799999   
                    21320  0.0    0.000  -71.029999  47.990002   
                    21323  0.0    0.001 -110.320000  32.939999   
                    21324  0.0    0.000  -95.980003  39.849998   
                    21328  0.0    0.000 -119.690002  38.070000   
                    21329  0.0    0.000  -75.889999  36.740002   
                    21330  0.0    0.002 -111.050003  33.299999   
                    21331  0.0    0.002 -110.879997  33.080002   
                    21332  0.0    0.005  -82.709999  34.650002   
                    21333  0.0    0.002 -110.239998  31.670000   
2022-01-01 00:02:44 20964  0.0    0.005  -83.870003  34.730000   
                    21312  0.0    0.004 -110.470001  32.799999   
                    21320  0.0    0.000  -71.029999  47.990002   
                    21323  0.0    0.001 -110.320000  32.939999   
                    21324  0.0    0.000  -95.980003  39.849998   
                    21328  0.0    0.000 -119.690002  38.070000   
                    21330  0.0    0.002 -111.050003  33.299999   
                    21331  0.0    0.002 -110.879997  33.080002   
                    21332  0.0    0.005  -82.680000  34.660000   
                    21333  0.0    0.002 -110.239998  31.670000   
                    21334  0.0    0.000  -80.970001  43.759998   
                    21335  0.0    0.003 -109.949997  32.130001   

                           MEANWIND_1-3kmAGL  MESH  ...  P98LLAZ  P98MLAZ  PS  \
time                ID                              ...                         
2022-01-01 00:00:38 20964          36.799999  0.04  ...    0.004    0.004   8   
                    21312          34.500000  0.00  ...    0.003    0.003   3   
                    21320          31.200001  0.18 